In [60]:
import subprocess
import folium
from geopy.geocoders import Nominatim
import requests

def trace_route(destination):
    """Функция для выполнения traceroute и получения траектории сетевых маршрутов.
    :param destination: Конечный сервер (IP или доменное имя)
    :return: Список хопов с временем ответа и их IP-адресами
    """
    command = ['tracert', '-h', '30', '-w', '200', '-d', destination]  # Для Windows
    result = subprocess.check_output(command, universal_newlines=True)
    return result

def get_geolocation(ip, service = 1):
    
    if service == 1:
        url = f"http://ip-api.com/json/{ip}"
    else:    
        url = f"http://ipwho.is/{ip}"
    
    response = requests.get(url)
    data = response.json()

    if service == 1:
        if data['status'] == 'success':
            return data['lat'], data['lon'], data['country'], data['city']
    else:
        if data['success'] == True:
            return data['latitude'], data['longitude'], data['country'], data['city']
    return None, None



def plot_traceroute_on_map(trace_output):
    """
    Функция для отображения результатов трассировки на карте.
    :param trace_output: Вывод команды traceroute
    """
    ips = []
    for line in trace_output.splitlines():
        if 'ms' in line:
            parts = line.split()
            ip = parts[len(parts)-1] if len(parts) > 2 else None
            if ip:
                ip = ip.replace("[", "").replace("]", "")
                ips.append(ip)

    # Инициализация карты с базовыми координатами
    m = folium.Map(location=[20, 0], zoom_start=2)
    
    latitudes = []
    longitudes = []
    coordinates = []

    # Добавляем маркеры на карту
    for ip in ips:
        lat, lon, _, _ = get_geolocation(ip)
        if lat and lon:
            # Сохраняем координаты
            coordinates.append((lat, lon))
            
            # Добавляем круговой маркер
            folium.CircleMarker(
                location=(lat, lon),
                radius=5,
                color='blue',
                fill=True,
                fill_color='blue',
                fill_opacity=0.6,
                popup=f'IP: {ip}'
            ).add_to(m)
    
    # Соединяем точки линиями
    if coordinates:
        folium.PolyLine(
            locations=coordinates,
            color="red",
            weight=2.5,
            opacity=0.8
        ).add_to(m)

    # Сохраняем карту
    m.save("traceroute_map.html")
    print("Карта с трассировкой сохранена как 'traceroute_map.html'")

    return coordinates, ips



In [50]:
# Пример использования
destination = "vk.com"  # Вы можете заменить на любой сервер
trace_output = trace_route(destination)
coord, ips = plot_traceroute_on_map(trace_output)

Карта с трассировкой сохранена как 'traceroute_map.html'


In [76]:
def safe_get(data, index, default=None):
    try:
        return data[index]
    except IndexError:
        return 

In [80]:
compare_dict = {}

for idx, ip in enumerate(ips):
    compare_dict[ip] = {}
    data = get_geolocation(ip)
    compare_dict[ip][1] = {
        'country': safe_get(data,2),
        'city': safe_get(data,3)
    }

    data = get_geolocation(ip,2)
    compare_dict[ip][2] = {
        'country': safe_get(data,2),
        'city': safe_get(data,3)
    }

In [81]:
compare_dict

{'192.168.1.1': {1: {'country': None, 'city': None},
  2: {'country': None, 'city': None}},
 '62.168.171.97': {1: {'country': 'Georgia', 'city': 'Tbilisi'},
  2: {'country': 'Georgia', 'city': 'Tbilisi'}},
 '213.157.194.2': {1: {'country': 'Georgia', 'city': 'Tbilisi'},
  2: {'country': 'Georgia', 'city': 'Tbilisi'}},
 '213.157.194.1': {1: {'country': 'Georgia', 'city': 'Tbilisi'},
  2: {'country': 'Georgia', 'city': 'Tbilisi'}},
 '84.44.20.157': {1: {'country': 'Türkiye', 'city': 'Istanbul'},
  2: {'country': 'Turkey', 'city': 'Istanbul'}},
 '195.2.23.129': {1: {'country': 'Türkiye', 'city': 'Tuzla'},
  2: {'country': 'Ireland', 'city': 'Dublin'}},
 '195.2.18.213': {1: {'country': 'Türkiye', 'city': 'Esenyurt'},
  2: {'country': 'United States', 'city': 'Stockton'}},
 '195.2.18.210': {1: {'country': 'Bulgaria', 'city': 'Sofia'},
  2: {'country': 'United States', 'city': 'Stockton'}},
 '62.115.36.50': {1: {'country': 'Bulgaria', 'city': 'Sofia'},
  2: {'country': 'Bulgaria', 'city': 'S

In [82]:
import pandas as pd

In [86]:
df = pd.DataFrame.from_dict({(i, j): compare_dict[i][j] 
                             for i in compare_dict.keys() 
                             for j in compare_dict[i].keys()},
                            orient='index')

# Преобразование индекса в MultiIndex
df.index = pd.MultiIndex.from_tuples(df.index, names=['IP', 'Instance'])

In [87]:
df

country               city
IP             Instance                                  
192.168.1.1    1                  None               None
               2                  None               None
62.168.171.97  1               Georgia            Tbilisi
               2               Georgia            Tbilisi
213.157.194.2  1               Georgia            Tbilisi
               2               Georgia            Tbilisi
213.157.194.1  1               Georgia            Tbilisi
               2               Georgia            Tbilisi
84.44.20.157   1               Türkiye           Istanbul
               2                Turkey           Istanbul
195.2.23.129   1               Türkiye              Tuzla
               2               Ireland             Dublin
195.2.18.213   1               Türkiye           Esenyurt
               2         United States           Stockton
195.2.18.210   1              Bulgaria              Sofia
               2         United States           Stockton
62.115.36.50   1              Bulgaria              Sofia
               2              Bulgaria              Sofia
62.115.123.36  1               Austria             Vienna
               2               Austria             Vienna
62.115.138.22  1               Germany  Frankfurt am Main
               2               Germany          Frankfurt
62.115.138.104 1               Germany  Frankfurt am Main
               2                Sweden          Stockholm
62.115.115.247 1         United States     St. Petersburg
               2                Russia   Saint Petersburg
62.115.145.97  1         United States     St. Petersburg
               2               Belgium           Brussels
87.240.137.164 1                Russia      St Petersburg
               2                Russia   Saint Petersburg

In [45]:
from geopy.distance import geodesic


def calculate_distances(coords):
    distances = []
    for i in range(len(coords) - 1):
        distance = geodesic(coords[i], coords[i + 1]).kilometers
        distances.append(distance)
    return distances

distances = calculate_distances(coord)
for i, distance in enumerate(distances):
    print(f"Distance between point {i + 1} and point {i + 2}: {distance:.2f} km")


Distance between point 1 and point 2: 0.00 km
Distance between point 2 and point 3: 0.00 km
Distance between point 3 and point 4: 1329.51 km
Distance between point 4 and point 5: 35.73 km
Distance between point 5 and point 6: 8005.48 km
Distance between point 6 and point 7: 9646.57 km
Distance between point 7 and point 8: 1043.11 km
Distance between point 8 and point 9: 599.08 km
Distance between point 9 and point 10: 0.00 km
Distance between point 10 and point 11: 7783.06 km
Distance between point 11 and point 12: 0.00 km
Distance between point 12 and point 13: 8547.46 km


In [46]:
round(sum(distances))

36990

1 - 36990
2 - 32410